# Assignment 3
## Econ 8310 - Business Forecasting

For homework assignment 3, you will work with [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist), a more fancier data set.

- You must create a custom data loader as described in the first week of neural network lectures [2 points]
    - You will NOT receive credit for this if you use the pytorch prebuilt loader for Fashion MNIST!
- You must create a working and trained neural network using only pytorch [2 points]
- You must store your weights and create an import script so that I can evaluate your model without training it [2 points]

Highest accuracy score gets some extra credit!

Submit your forked repository URL on Canvas! :) I'll be manually grading this assignment.

Some checks you can make on your own:
- Did you manually process the data or use a prebuilt loader (see above)?
- Does your script train a neural network on the assigned data?
- Did your script save your model?
- Do you have separate code to import your model for use after training?

In [15]:
import os
import gzip
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Load image data from Fashion MNIST gzip file
def load_images(path):
    with gzip.open(path, 'rb') as f:
        _ = int.from_bytes(f.read(4), 'big')  # magic number
        num_images = int.from_bytes(f.read(4), 'big')
        rows = int.from_bytes(f.read(4), 'big')
        cols = int.from_bytes(f.read(4), 'big')
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8).reshape(num_images, 1, rows, cols)
        return images / 255.0  # Normalize to [0, 1]

# Load label data from Fashion MNIST gzip file
def load_labels(path):
    with gzip.open(path, 'rb') as f:
        _ = int.from_bytes(f.read(4), 'big')  # magic number
        num_labels = int.from_bytes(f.read(4), 'big')
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

class FashionMNISTDataset(Dataset):
    def __init__(self, images_path, labels_path):
        self.images = torch.tensor(load_images(images_path), dtype=torch.float32)
        self.labels = torch.tensor(load_labels(labels_path), dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

import os

#Struggled with this section so this is a HUGE help from AI
# Create folder if it doesn't exist.
os.makedirs("data/fashion", exist_ok=True)

# Download raw .gz files from GitHub
!wget -O data/fashion/train-images-idx3-ubyte.gz https://raw.githubusercontent.com/kelleespringer/econ8310-assignment3/main/data/train-images-idx3-ubyte.gz
!wget -O data/fashion/train-labels-idx1-ubyte.gz https://raw.githubusercontent.com/kelleespringer/econ8310-assignment3/main/data/train-labels-idx1-ubyte.gz
!wget -O data/fashion/t10k-images-idx3-ubyte.gz https://raw.githubusercontent.com/kelleespringer/econ8310-assignment3/main/data/t10k-images-idx3-ubyte.gz
!wget -O data/fashion/t10k-labels-idx1-ubyte.gz https://raw.githubusercontent.com/kelleespringer/econ8310-assignment3/main/data/t10k-labels-idx1-ubyte.gz

# Instantiate dataset using LOCAL file paths
train_dataset = FashionMNISTDataset(
    'data/fashion/train-images-idx3-ubyte.gz',
    'data/fashion/train-labels-idx1-ubyte.gz'
)


# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


--2025-04-15 00:38:13--  https://raw.githubusercontent.com/kelleespringer/econ8310-assignment3/main/data/train-images-idx3-ubyte.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26421880 (25M) [application/octet-stream]
Saving to: ‘data/fashion/train-images-idx3-ubyte.gz’

data/fashion/train- 100%[===================>]  25.20M  --.-KB/s    in 0.1s    

2025-04-15 00:38:13 (172 MB/s) - ‘data/fashion/train-images-idx3-ubyte.gz’ saved [26421880/26421880]

--2025-04-15 00:38:13--  https://raw.githubusercontent.com/kelleespringer/econ8310-assignment3/main/data/train-labels-idx1-ubyte.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubuser

In [20]:
import torch.nn as nn
import torch.nn.functional as F

class FashionClassifier(nn.Module):
    def __init__(self):
        super(FashionClassifier, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = FashionClassifier().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")




Epoch 1, Loss: 0.5545
Epoch 2, Loss: 0.3940
Epoch 3, Loss: 0.3537
Epoch 4, Loss: 0.3251
Epoch 5, Loss: 0.3075


In [30]:
# Save model weights
torch.save(model.state_dict(), 'fashion_mnist_model.pth')
print("Model saved!")

Model saved!


In [31]:
def load_model(model_path):
    model = FashionClassifier()
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set to evaluation mode
    return model

model = load_model('fashion_mnist_model.pth')


In [32]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to load model
def load_model(model_path):
    model = FashionClassifier()
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Evaluate the model
def evaluate(model, test_loader):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy

# Main function to run evaluation
if __name__ == "__main__":
    # Set up test loader
    test_dataset = FashionMNISTDataset(
        'data/fashion/t10k-images-idx3-ubyte.gz',
        'data/fashion/t10k-labels-idx1-ubyte.gz'
    )
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Load the trained model
    model = load_model('fashion_mnist_model.pth')

    # Evaluate the model
    evaluate(model, test_loader)


Test Accuracy: 86.92%
